In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import csv
import itertools
import collections
import pywt
from scipy import stats
from scipy.signal import butter, lfilter, freqz
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, AvgPool1D, Flatten, Dense, Dropout, Softmax
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
plt.rcParams["figure.figsize"] = (30,6)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.color'] = 'b'
plt.rcParams['axes.grid'] = True

In [4]:
filepath = 'mitbih_database/'
window_size = 180
maximum_counting = 10000

classnames = ['N', 'L', 'R', 'A', 'V']
n_classes = len(classnames)
count_classes = [0]*n_classes

X = list()
y = list()

In [ ]:

filenames = next(os.walk(filepath))[2]
signaldata = list()
annotations = list()
filenames.sort()
print((filenames))

In [ ]:
for f in filenames:
    filename, file_ext = os.path.splitext(f)

    if(file_ext == '.csv'):
        signaldata.append(filepath + filename + file_ext)

    elif(file_ext == '.txt'):
        annotations.append(filepath + filename + file_ext)


In [ ]:
window_size = 180
for r in range(0, len(signaldata)):
    signals = []

    with open(signaldata[r], 'rt',) as csvfile:
        tempstorage = csv.reader(csvfile, delimiter=',',
                                 quotechar='|')
        insrtindx = -1
        for row in tempstorage:
            if(insrtindx >= 0):
                signals.insert(insrtindx, int(row[1]))
            insrtindx = insrtindx+1

    signals = stats.zscore(signals)

    if r == 1:
        plt_1 = plt.figure(figsize=(30, 6))
        plt.grid()

        plt.title(signaldata[1] + " Ecg Wave")

        plt.plot(signals[0:700])
        plt.show()
        sc=signals

    boolvar = False
    with open(annotations[r], 'r') as fileID:
        data = fileID.readlines()
        beat = list()

        for d in range(1, len(data)):
            splitted = data[d].split(' ')
            splitted = filter(None, splitted)
            next(splitted)
            pos = int(next(splitted))
            class_type = next(splitted)
            if(class_type in classnames):
                c_ind = classnames.index(class_type)

                count_classes[c_ind] += 1
                if(window_size <= pos and pos < (len(signals) - window_size)):
                    beat = signals[pos-window_size:pos+window_size]
                    if r == 1 and not boolvar:
                        plt_1 = plt.figure(figsize=(30, 6))
                        plt.grid()
                        plt.title("A Beat from " + signaldata[1] + " Ecg Wave")
                        plt.plot(beat)
                        plt.show()
                        boolvar = True

                    X.append(beat)
                    y.append(c_ind)


In [ ]:
for i in range(0,len(X)):
        X[i] = np.append(X[i], y[i])

print(np.shape(X))


In [ ]:
X_train_df = pd.DataFrame(X)
per_class = X_train_df[X_train_df.shape[1]-1].value_counts()
print(per_class)
plt.figure(figsize=(20,10))
my_circle=plt.Circle( (0,0), 0.7, color='white')
explode=(0,0.1,0.1,0.1,0.1)
plt.pie(per_class, labels=['Non-ectopic Beats', 'Left Bundle Branch Block', 'Right Bundle Branch Block', 'Atrial Premature Contraction', 'Premature Ventricular Contraction'],explode=explode, colors=['tab:blue','tab:orange','tab:purple','tab:red','tab:green'],autopct='%1.1f%%')
plt.show()

In [ ]:
target=X_train_df.shape[1]-1
df_0=X_train_df[X_train_df[target]==0]
df_1=X_train_df[X_train_df[target]==1]
df_2=X_train_df[X_train_df[target]==2]
df_3=X_train_df[X_train_df[target]==3]
df_4=X_train_df[X_train_df[target]==4]

In [ ]:
countclassno=2500



df_0_undersample=df_0.sample(countclassno,replace=True)

df_1_undersample=df_1.sample(countclassno,replace=True)
df_2_undersample=df_2.sample(countclassno,replace=True)
df_3_undersample=df_3.sample(countclassno,replace=True)
df_4_undersample=df_4.sample(countclassno,replace=True)

X_train_df=pd.concat([df_0_undersample,df_1_undersample,df_2_undersample,df_3_undersample,df_4_undersample])

In [ ]:
class_dist = X_train_df[X_train_df.shape[1]-1].value_counts()
print(class_dist)
plt.figure(figsize=(20,10))

plt.pie(class_dist, labels=['-N : Non-ectopic beats (normal beat) ', '-L : Left Bundle Branch Block ', '-R : Right Bundle Branch Block ', '-A : Atrial Premature Contraction ', '-V : Premature Ventricular Contraction'], colors=['tab:blue','tab:orange','tab:purple','tab:red','tab:green'],autopct='%1.1f%%')

plt.show()

In [ ]:
train, test = train_test_split(X_train_df, test_size=0.20,random_state=45637)

print("X_train : ", np.shape(train))
print("X_test  : ", np.shape(test))

In [ ]:
target_train=train[train.shape[1]-1]
target_test=test[test.shape[1]-1]
train_y=to_categorical(target_train)
test_y=to_categorical(target_test)
print(np.shape(train_y), np.shape(test_y))

In [ ]:
train_x = train.iloc[:,:train.shape[1]-1].values
test_x = test.iloc[:,:test.shape[1]-1].values
train_x = train_x.reshape(len(train_x), train_x.shape[1],1)
test_x = test_x.reshape(len(test_x), test_x.shape[1],1)
print(np.shape(train_x), np.shape(test_x))